In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/preprocessed_data.csv")

In [ ]:
df.head()

,place,depth,mag,date,latitude,longitude
0,Wyoming,0.0,3.1,2013-06-01,43.731797,-105.600745
1,Alaska,26.0,3.1,2013-06-01,55.570312,-144.595396
2,Solomon Islands,58.8,4.7,2013-06-01,-10.610103,162.911383
3,Papua New Guinea,35.0,4.0,2013-05-31,-5.567791,150.321531
4,Tonga,101.9,4.8,2013-05-31,-18.905670,-174.892752


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198550 entries, 0 to 198549
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   place      198550 non-null  object 
 1   depth      198550 non-null  float64
 2   mag        198550 non-null  float64
 3   date       198550 non-null  object 
 4   latitude   198550 non-null  float64
 5   longitude  198550 non-null  float64
dtypes: float64(4), object(2)
memory usage: 9.1+ MB


In [ ]:
print(set(df['place']))

{'Mauritania', 'Sao Tome and Principe', 'Italy', 'Arizona', 'Kuwait', 'Zimbabwe', 'Papua New Guinea', 'Panama', 'Portugal', 'Czechia', 'Egypt', 'Vietnam', 'Burundi', 'Armenia', 'Barbados', 'Cayman Islands', 'Nebraska', 'India', 'New Jersey', 'Trinidad and Tobago', 'Saint Kitts and Nevis', 'Guadeloupe', 'Zambia', 'Connecticut', 'B.C.', 'Bhutan', 'Niger', 'Kentucky', 'North Carolina', 'Portugal region', 'Cuba', 'Wallis and Futuna', 'Madagascar', 'Sulawesi', 'Ghana', 'Laos', 'Botswana', 'Taiwan', 'Russia region', 'Nevada', 'Russia', 'Palau', 'WA', 'Jersey', 'Sakha', 'Brazil', 'Alaska region', 'Alaska Earthquake', 'Montserrat', 'Australia', 'Albania', 'OR', 'Maldives', 'Minnesota', 'Surigao del Norte', 'AK', 'Bosnia and Herzegovina', 'French Polynesia', 'Switzerland', 'Saint Helena', 'French Southern Territories', 'Japan Earthquake', 'Revilla Gigedo Islands', 'Virginia', 'Saint Vincent and the Grenadines', 'Uzbekistan', 'Myanmar', 'Tajikistan', 'Sweden', 'Texas Earthquake', 'Laikit II (Dim

In [ ]:
df['place'].nunique()

268

In [ ]:
eq_tmp = df.copy()

#rolling window size
DAYS_OUT_TO_PREDICT = 7

# loop through each zone and apply MA
eq_data = []
eq_data_last_days_out = []

for place in list(set(eq_tmp['place'])):
    temp_df = eq_tmp[eq_tmp['place'] == place].copy()

    #avg. depth of 22 days rolling period and so on..
    temp_df['depth_avg_22'] = temp_df['depth'].rolling(window=22,center=False).mean()
    temp_df['depth_avg_15'] = temp_df['depth'].rolling(window=15,center=False).mean()
    temp_df['depth_avg_7'] = temp_df['depth'].rolling(window=7,center=False).mean()
    temp_df['mag_avg_22'] = temp_df['mag'].rolling(window=22,center=False).mean()
    temp_df['mag_avg_15'] = temp_df['mag'].rolling(window=15,center=False).mean()
    temp_df['mag_avg_7'] = temp_df['mag'].rolling(window=7,center=False).mean()
    temp_df.loc[:, 'mag_outcome'] = temp_df.loc[:, 'mag_avg_7'].shift(DAYS_OUT_TO_PREDICT * -1)

    #days to predict value on earth quake data this is not yet seen or witnessed by next 7 days (consider as live next 7 days period)

    eq_data_last_days_out.append(temp_df.tail(DAYS_OUT_TO_PREDICT))

    eq_data.append(temp_df)

In [ ]:
# concat all location-based dataframes into master dataframe
eq_all = pd.concat(eq_data)

In [ ]:
eq_all.head()

,place,depth,mag,date,latitude,longitude,depth_avg_22,depth_avg_15,depth_avg_7,mag_avg_22,mag_avg_15,mag_avg_7,mag_outcome
938,Mauritania,13.80,4.2,2013-05-09,22.739312,-12.066524,NaN,NaN,NaN,NaN,NaN,NaN,4.171429
3211,Mauritania,10.00,4.1,2013-02-28,22.739312,-12.066524,NaN,NaN,NaN,NaN,NaN,NaN,4.214286
5447,Mauritania,10.00,4.2,2013-01-08,22.739312,-12.066524,NaN,NaN,NaN,NaN,NaN,NaN,4.257143
10028,Mauritania,15.01,4.1,2014-03-18,22.739312,-12.066524,NaN,NaN,NaN,NaN,NaN,NaN,4.257143
10384,Mauritania,14.79,4.0,2014-03-11,22.739312,-12.066524,NaN,NaN,NaN,NaN,NaN,NaN,4.257143


In [ ]:
import numpy as np
# remove any NaN fields
eq_all = eq_all[np.isfinite(eq_all['depth_avg_22'])]
eq_all = eq_all[np.isfinite(eq_all['mag_avg_22'])]
eq_all = eq_all[np.isfinite(eq_all['mag_outcome'])]

In [ ]:
eq_all.head()

,place,depth,mag,date,latitude,longitude,depth_avg_22,depth_avg_15,depth_avg_7,mag_avg_22,mag_avg_15,mag_avg_7,mag_outcome
17435,Italy,10.10,3.3,2015-03-04,41.657691,13.549476,52.355455,70.554667,78.654286,4.154545,4.140000,4.157143,4.314286
17589,Italy,17.30,4.2,2015-02-28,41.657691,13.549476,52.673636,67.328000,80.840000,4.190909,4.100000,4.257143,4.342857
18030,Italy,7.20,3.9,2015-02-18,41.657691,13.549476,52.637273,51.389333,76.154286,4.195455,4.086667,4.157143,4.371429
18511,Italy,245.41,4.8,2015-02-06,41.657691,13.549476,63.428636,57.859333,80.498571,4.263636,4.126667,4.171429,4.285714
18923,Italy,6.00,4.4,2015-01-30,41.657691,13.549476,62.296818,58.126000,78.370000,4.259091,4.153333,4.157143,4.300000


In [ ]:
eq_all.shape

(193709, 13)

In [ ]:
eq_data_last_days_out = pd.concat(eq_data_last_days_out)


eq_data_last_days_out = eq_data_last_days_out[np.isfinite(eq_data_last_days_out['mag_avg_22'])]
predict_unknown=eq_data_last_days_out

In [ ]:
# here 'mag_outcome' has NaN because these are future outcome event to be predicted live or data that has not yet been witnessed
predict_unknown

,place,depth,mag,date,latitude,longitude,depth_avg_22,depth_avg_15,depth_avg_7,mag_avg_22,mag_avg_15,mag_avg_7,mag_outcome
191334,Italy,10.000,4.2,2024-11-29,41.657691,13.549476,88.096955,64.526667,66.537143,4.140909,4.106667,4.071429,NaN
191890,Italy,265.157,4.3,2024-11-15,41.657691,13.549476,99.695000,54.400200,79.585571,4.140909,4.113333,4.114286,NaN
192154,Italy,10.000,4.0,2024-11-09,41.657691,13.549476,99.849273,53.694800,79.585571,4.136364,4.120000,4.114286,NaN
192670,Italy,10.000,4.0,2024-10-28,41.657691,13.549476,81.691227,53.722867,75.513571,4.109091,4.160000,4.100000,NaN
194159,Italy,10.000,3.9,2024-09-21,41.657691,13.549476,65.981227,53.722867,46.451000,4.090909,4.080000,4.057143,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139466,North Macedonia,95.790,4.4,2019-11-15,41.484421,21.118935,12.369091,14.672667,22.255714,4.054545,4.220000,4.214286,NaN
148249,North Macedonia,10.000,4.2,2020-12-02,41.484421,21.118935,12.369091,14.459333,22.255714,4.100000,4.160000,4.214286,NaN
149181,North Macedonia,10.000,4.9,2020-11-11,41.484421,21.118935,12.369091,14.879333,22.255714,4.122727,4.220000,4.314286,NaN
149186,North Macedonia,10.000,4.1,2020-11-11,41.484421,21.118935,12.369091,14.879333,22.255714,4.100000,4.213333,4.214286,NaN


In [ ]:
eq_all['mag_outcome'] = np.where(eq_all['mag_outcome'] > 4.3, 1,0)
print(eq_all['mag_outcome'].describe())
eq_all['mag_outcome'].value_counts()

count    193709.000000
mean          0.588078
std           0.492182
min           0.000000
25%           0.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: mag_outcome, dtype: float64


,count
mag_outcome,
1,113916
0,79793


In [ ]:
type(eq_all)

pandas.core.frame.DataFrame

In [ ]:
'''
eq_all.to_csv('Earthquake_features.csv', index=False)

# Download the file to your local system
from google.colab import files
files.download('Earthquake_features.csv')
'''

"\neq_all.to_csv('Earthquake_features.csv', index=False)\n\n# Download the file to your local system\nfrom google.colab import files\nfiles.download('Earthquake_features.csv')\n"

In [ ]:
'''
predict_unknown.to_csv('Earthquake_predict.csv', index=False)

# Download the file to your local system
from google.colab import files
files.download('Earthquake_predict.csv')
'''

"\npredict_unknown.to_csv('Earthquake_predict.csv', index=False)\n\n# Download the file to your local system\nfrom google.colab import files\nfiles.download('Earthquake_predict.csv')\n"

In [ ]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:////content/drive/MyDrive/Earthquakedata.db')
eq_all.to_sql('Earthquake_features', engine, index=False, if_exists='replace')

193709

In [ ]:
predict_unknown.to_sql('Earthquake_predict', engine, index=False,if_exists='replace')

992

In [ ]:
from sqlalchemy import inspect
inspector = inspect(engine)
print(inspector.get_table_names())

['Earthquake_features', 'Earthquake_predict']


In [ ]:
df_features = pd.read_sql_table('Earthquake_features',con=engine)
df_features.head()

,place,depth,mag,date,latitude,longitude,depth_avg_22,depth_avg_15,depth_avg_7,mag_avg_22,mag_avg_15,mag_avg_7,mag_outcome
0,Italy,10.10,3.3,2015-03-04,41.657691,13.549476,52.355455,70.554667,78.654286,4.154545,4.140000,4.157143,1
1,Italy,17.30,4.2,2015-02-28,41.657691,13.549476,52.673636,67.328000,80.840000,4.190909,4.100000,4.257143,1
2,Italy,7.20,3.9,2015-02-18,41.657691,13.549476,52.637273,51.389333,76.154286,4.195455,4.086667,4.157143,1
3,Italy,245.41,4.8,2015-02-06,41.657691,13.549476,63.428636,57.859333,80.498571,4.263636,4.126667,4.171429,0
4,Italy,6.00,4.4,2015-01-30,41.657691,13.549476,62.296818,58.126000,78.370000,4.259091,4.153333,4.157143,0


In [ ]:
df_predict = pd.read_sql_table('Earthquake_predict',con=engine)
df_predict.head()

,place,depth,mag,date,latitude,longitude,depth_avg_22,depth_avg_15,depth_avg_7,mag_avg_22,mag_avg_15,mag_avg_7,mag_outcome
0,Italy,10.000,4.2,2024-11-29,41.657691,13.549476,88.096955,64.526667,66.537143,4.140909,4.106667,4.071429,NaN
1,Italy,265.157,4.3,2024-11-15,41.657691,13.549476,99.695000,54.400200,79.585571,4.140909,4.113333,4.114286,NaN
2,Italy,10.000,4.0,2024-11-09,41.657691,13.549476,99.849273,53.694800,79.585571,4.136364,4.120000,4.114286,NaN
3,Italy,10.000,4.0,2024-10-28,41.657691,13.549476,81.691227,53.722867,75.513571,4.109091,4.160000,4.100000,NaN
4,Italy,10.000,3.9,2024-09-21,41.657691,13.549476,65.981227,53.722867,46.451000,4.090909,4.080000,4.057143,NaN


In [ ]:
# Write a SQL query to select data from the Earthquake_features table
query = "SELECT * FROM Earthquake_features LIMIT 10"  # Selecting first 10 rows as an example

# Execute the query using the engine and read into a DataFrame
selected_data = pd.read_sql(query, con=engine)

# Display the selected data
display(selected_data)

,place,depth,mag,date,latitude,longitude,depth_avg_22,depth_avg_15,depth_avg_7,mag_avg_22,mag_avg_15,mag_avg_7,mag_outcome
0,Italy,10.10,3.3,2015-03-04,41.657691,13.549476,52.355455,70.554667,78.654286,4.154545,4.140000,4.157143,1
1,Italy,17.30,4.2,2015-02-28,41.657691,13.549476,52.673636,67.328000,80.840000,4.190909,4.100000,4.257143,1
2,Italy,7.20,3.9,2015-02-18,41.657691,13.549476,52.637273,51.389333,76.154286,4.195455,4.086667,4.157143,1
3,Italy,245.41,4.8,2015-02-06,41.657691,13.549476,63.428636,57.859333,80.498571,4.263636,4.126667,4.171429,0
4,Italy,6.00,4.4,2015-01-30,41.657691,13.549476,62.296818,58.126000,78.370000,4.259091,4.153333,4.157143,0
5,Italy,9.70,4.5,2015-01-23,41.657691,13.549476,62.251364,58.039333,43.558571,4.245455,4.186667,4.157143,0
6,Italy,281.68,4.2,2015-01-02,41.657691,13.549476,74.350455,76.018000,82.484286,4.213636,4.200000,4.185714,0
7,Italy,318.14,4.2,2015-01-02,41.657691,13.549476,88.352273,96.560667,126.490000,4.195455,4.186667,4.314286,1
8,Italy,15.00,4.4,2016-05-31,41.657691,13.549476,86.047727,96.734000,126.161429,4.177273,4.246667,4.342857,0
9,Italy,19.00,4.1,2016-05-26,41.657691,13.549476,75.716818,97.867333,127.847143,4.177273,4.286667,4.371429,0


In [ ]:
df_dashboard = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dashboard_data.csv")
df_dashboard.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,date
0,2013-05-31T23:58:13.560Z,43.304,-105.404,0.0,3.1,ml,10.0,141.5,NaN,1.89,...,Wyoming,explosion,NaN,NaN,NaN,NaN,reviewed,us,us,2013-06-01
1,2013-05-31T22:34:01.820Z,51.127,-178.080,26.0,3.1,ml,17.0,261.5,NaN,NaN,...,Alaska,earthquake,NaN,NaN,NaN,NaN,reviewed,aeic,aeic,2013-06-01
2,2013-05-31T22:21:55.750Z,21.648,143.032,322.5,4.4,mb,89.0,40.0,NaN,0.55,...,NaN,earthquake,NaN,4.5,NaN,86.0,reviewed,us,us,2013-06-01
3,2013-05-31T20:17:11.020Z,-10.280,161.560,58.8,4.7,mb,48.0,105.0,NaN,0.92,...,Solomon Islands,earthquake,NaN,6.3,NaN,40.0,reviewed,us,us,2013-06-01
4,2013-05-31T17:36:02.130Z,-43.360,172.945,20.3,4.0,ml,21.0,100.0,NaN,NaN,...,NaN,earthquake,NaN,NaN,NaN,NaN,reviewed,wel,wel,2013-05-31


In [ ]:
df_dashboard.to_sql('dashboard_data', engine, index=False,if_exists='replace')

238992

In [ ]:
df_dash = pd.read_sql_table('dashboard_data',con=engine)
df_dash.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,date
0,2013-05-31T23:58:13.560Z,43.304,-105.404,0.0,3.1,ml,10.0,141.5,NaN,1.89,...,Wyoming,explosion,NaN,NaN,NaN,NaN,reviewed,us,us,2013-06-01
1,2013-05-31T22:34:01.820Z,51.127,-178.080,26.0,3.1,ml,17.0,261.5,NaN,NaN,...,Alaska,earthquake,NaN,NaN,NaN,NaN,reviewed,aeic,aeic,2013-06-01
2,2013-05-31T22:21:55.750Z,21.648,143.032,322.5,4.4,mb,89.0,40.0,NaN,0.55,...,None,earthquake,NaN,4.5,NaN,86.0,reviewed,us,us,2013-06-01
3,2013-05-31T20:17:11.020Z,-10.280,161.560,58.8,4.7,mb,48.0,105.0,NaN,0.92,...,Solomon Islands,earthquake,NaN,6.3,NaN,40.0,reviewed,us,us,2013-06-01
4,2013-05-31T17:36:02.130Z,-43.360,172.945,20.3,4.0,ml,21.0,100.0,NaN,NaN,...,None,earthquake,NaN,NaN,NaN,NaN,reviewed,wel,wel,2013-05-31
